## Основы Python, numPy, pandas на примере данных Auto.ru

<font color=darkblue>Импортируем нужные пакеты

In [21]:
import pandas as pd
import numpy as np
import re
from datetime import date as Date
from numpy import nan as NaN

%matplotlib inline

<font color=darkblue>Загрузка данных из csv-файла в датафрейм

In [22]:
df = pd.read_csv(u'../data/auto.ru_data.csv')

In [25]:
df.head(5)

,Год выпуска,КПП,Привод,Пробег,Тип кузова,Состояние,Срок владения,Цвет,Руль,Владельцы,...,Модель,Город,Объем двигателя,Мощность двигателя,Двигатель,Цена,Дата,Количество дней,Просмотров всего,Просмотров в день
0,2012.0,механическая,передний,141000.0,внедорожник 5 дв.,Не требует ремонта,NaN,белый,Левый,2 владельца,...,ix35 I,Воронеж,2.0,150.0,Бензин,790000,2018-06-25,60,387.0,6.450000
1,2014.0,автоматическая,полный,80189.0,внедорожник 5 дв.,Не требует ремонта,NaN,белый,Левый,3 или более,...,X1 I (E84) Рестайлинг 20d,Москва,2.0,184.0,Дизель,1368000,2018-06-25,60,657.0,10.950000
2,2010.0,механическая,передний,143000.0,седан,Не требует ремонта,NaN,бежевый,Левый,3 или более,...,Logan I Рестайлинг,Воронеж,1.4,75.0,Бензин,320000,2018-06-25,60,470.0,7.833333
3,2009.0,механическая,передний,126000.0,седан,Не требует ремонта,NaN,жёлтый,Левый,3 или более,...,Nexia I Рестайлинг,Россошь,1.5,80.0,Бензин,150000,2018-06-25,60,438.0,7.300000
4,2011.0,механическая,передний,159000.0,хэтчбек 5 дв.,Не требует ремонта,NaN,красный,Левый,3 или более,...,C4 II,Воронеж,1.6,120.0,Бензин,410000,2018-06-25,60,442.0,7.366667


In [4]:
df.columns

Index(['Год выпуска', 'КПП', 'Привод', 'Пробег', 'Тип кузова', 'Состояние',
       'Срок владения', 'Цвет', 'Руль', 'Владельцы', 'ПТС', 'Комментарий',
       'Марка', 'Модель', 'Город', 'Объем двигателя', 'Мощность двигателя',
       'Двигатель', 'Цена', 'Дата', 'Количество дней', 'Просмотров всего',
       'Просмотров в день'],
      dtype='object')

In [5]:
df.head()

,Год выпуска,КПП,Привод,Пробег,Тип кузова,Состояние,Срок владения,Цвет,Руль,Владельцы,...,Модель,Город,Объем двигателя,Мощность двигателя,Двигатель,Цена,Дата,Количество дней,Просмотров всего,Просмотров в день
0,2012.0,механическая,передний,141000.0,внедорожник 5 дв.,Не требует ремонта,NaN,белый,Левый,2 владельца,...,ix35 I,Воронеж,2.0,150.0,Бензин,790000,2018-06-25,60,387.0,6.450000
1,2014.0,автоматическая,полный,80189.0,внедорожник 5 дв.,Не требует ремонта,NaN,белый,Левый,3 или более,...,X1 I (E84) Рестайлинг 20d,Москва,2.0,184.0,Дизель,1368000,2018-06-25,60,657.0,10.950000
2,2010.0,механическая,передний,143000.0,седан,Не требует ремонта,NaN,бежевый,Левый,3 или более,...,Logan I Рестайлинг,Воронеж,1.4,75.0,Бензин,320000,2018-06-25,60,470.0,7.833333
3,2009.0,механическая,передний,126000.0,седан,Не требует ремонта,NaN,жёлтый,Левый,3 или более,...,Nexia I Рестайлинг,Россошь,1.5,80.0,Бензин,150000,2018-06-25,60,438.0,7.300000
4,2011.0,механическая,передний,159000.0,хэтчбек 5 дв.,Не требует ремонта,NaN,красный,Левый,3 или более,...,C4 II,Воронеж,1.6,120.0,Бензин,410000,2018-06-25,60,442.0,7.366667


<font color=darkblue>Добавим столбец с возрастом авто

In [6]:
df["Возраст"] = 2018 - df["Год выпуска"]

### Диаграмма sankey. КПП - Привод

In [7]:
import matplotlib
import matplotlib.pyplot as plt

import plotly
import plotly.plotly as py

from plotly.offline import init_notebook_mode, iplot
init_notebook_mode(connected=True)

import random
r = lambda: random.randint(0,255)

In [8]:
def minimize_data_by_mark(df_in, auto_min_cnt):
    #Отсечем данные по популярным маркам в продаже
    df_temp = df_in.groupby(['Марка']).count().reset_index()
    mark_list = df_temp[(df_temp['Год выпуска'] > auto_min_cnt)]['Марка'].tolist()
    df_res = df_in[(df_in['Марка'].isin(mark_list))]
    
    return df_res

<font color=darkblue>Функция для получения датафрейма с данными для plotly sunkey чарта

In [9]:
def prepare_sankey_data(df_in, attribute_list):
    labels = pd.DataFrame()
    for attr in attribute_list:
        label = pd.DataFrame(df_in[attr].unique(), columns=['label']).reset_index(drop=True)
        labels = pd.DataFrame(pd.concat([labels, label], axis=0))

    labels['color'] = labels['label'].apply(lambda x: ('#%02X%02X%02X' % (r(),r(),r())))
    labels = labels.reset_index(drop=True)
    labels['ID'] = labels.index

    df_res = pd.DataFrame()
    for i, attr in enumerate(attribute_list):
        if (i == len(attribute_list)-1):
            break
        df_tmp = df_in.groupby([attribute_list[i], attribute_list[i+1]]).count().reset_index()
        df_tmp = pd.merge(df_tmp, labels, how='left', left_on=[attr], right_on=['label'])
        df_tmp = pd.merge(df_tmp, labels, how='left', left_on=[attribute_list[i+1]], right_on=['label'])
        df_tmp = df_tmp[['ID_x','ID_y', 'Год выпуска']]
        df_tmp.columns = ['source_value', 'dest_value', 'amount']
        df_res = pd.concat([df_res, df_tmp], axis=0)
    
    return labels, df_res

<font color=darkblue>Функция для вормирования справочника для sankey чарта

In [10]:
def draw_sankey(labels, df_res, title):
    data_trace = dict(
        type='sankey',
        domain = dict(
          x =  [0,1],
          y =  [0,1]
        ),
        orientation = "h",
        valueformat = ".0f",
        node = dict(
            pad = 10,
            thickness = 30,
            line = dict(
            color = "black",
            width = 0.5
          ),
        label = labels['label'].tolist(),
        color = labels['color'].tolist(),
        ),
        link = dict(
        source = df_res['source_value'].tolist(),
        target = df_res['dest_value'].tolist(),
        value = df_res['amount'].tolist(),
        ),
    )

    layout =  dict(
        title = title,
        height = 772,
        width = 950,
        font = dict(
            size = 14
        ),    
    )
    
    fig = dict(data=[data_trace], layout=layout)
    iplot(fig, validate=False)

<font color=darkblue>Подготовим датасет
Разобъем пробег по классам

In [11]:
df['Пробег тип'] = ''
df.loc[df.Пробег < 20000, 'Пробег тип'] = '0-20 тыс.'
df.loc[(df.Пробег >= 20000) & (df.Пробег < 40000), 'Пробег тип'] = '20-40 тыс.'
df.loc[(df.Пробег >= 40000) & (df.Пробег < 60000), 'Пробег тип'] = '40-60 тыс.'
df.loc[(df.Пробег >= 60000) & (df.Пробег < 80000), 'Пробег тип'] = '60-80 тыс.'
df.loc[(df.Пробег >= 80000) & (df.Пробег < 100000), 'Пробег тип'] = '80-100 тыс.'
df.loc[df.Пробег >= 100000, 'Пробег тип'] = 'более 100 тыс.'

<font color=darkblue>Также по количеству владельцев

In [12]:
df['Количество владельцев'] = df['Владельцы'].apply(lambda x: ("Владельцев: %s" % str(x)))

In [13]:
#df.loc[df['Просмотров в день'] == 'более 60'] = 60
df['Просмотров в день тип'] = ''
df.loc[df['Просмотров в день'] < 10, 'Просмотров в день тип'] = '0-10 просмотра'
df.loc[(df['Просмотров в день'] >=10) & (df['Просмотров в день'] < 20), 'Просмотров в день тип'] = '10-20 просмотра'
df.loc[(df['Просмотров в день'] >= 20) & (df['Просмотров в день'] < 30), 'Просмотров в день тип'] = '20-30 просмотра'
df.loc[(df['Просмотров в день'] >= 30) & (df['Просмотров в день'] < 40), 'Просмотров в день тип'] = '30-40 просмотра'
df.loc[(df['Просмотров в день'] >= 40) & (df['Просмотров в день'] < 50), 'Просмотров в день тип'] = '40-50 просмотра'
df.loc[(df['Просмотров в день'] >= 50) & (df['Просмотров в день'] < 60), 'Просмотров в день тип'] = '50-60 просмотра'
df.loc[(df['Просмотров в день'] >= 60), 'Просмотров в день тип'] = 'более 60'

In [14]:
labels, df_res = prepare_sankey_data(df, ['КПП', 'Привод'])

In [15]:
draw_sankey(labels, df_res, 'Большинство полноприводных машин с автоматической КПП<br>Большинство машин с механической КПП - переднеприводные')

In [16]:
df_temp = df[(df['Цвет'] == 'красный')]
df_temp = minimize_data_by_mark(df_temp, 20)
labels, df_res = prepare_sankey_data(df_temp, ['Марка', 'Тип кузова'])
draw_sankey(labels, df_res, 'Красные Nissan в основном внедорожники<br>красные Ford и Renault в основном хэтчбэки<br>красные Mazda и Audi в основном седаны')

In [17]:
labels, df_res = prepare_sankey_data(df, ['Двигатель', 'Привод'])
draw_sankey(labels, df_res, 'Заднеприводные авто в осноаном с бензиновыми двигателями<br>Дизельные машины в основном полноприводные')

In [18]:
df_temp = minimize_data_by_mark(df, 800)
df_temp = df_temp[(df_temp['Пробег'] < 100000)]
df_temp = df_temp[(df_temp['Владельцы'] == '1 владелец')]
labels, df_res = prepare_sankey_data(df_temp, ['Марка', 'Пробег тип'])
draw_sankey(labels, df_res, 'При каком пробеге владелец (купивший авто с нуля) хочет продать <br> свою машину в зависимости от марки')

<font color=darkblue>Быстрее всего хотят продать Lada, дольше ездить хотят на Toyota и Volkswagen

In [19]:
df_temp = minimize_data_by_mark(df, 700)
labels, df_res = prepare_sankey_data(df_temp, ['Марка', 'Просмотров в день тип'])
draw_sankey(labels, df_res, 'Марка - Просмотров в день (интерес к марке)')

<font color=darkblue>Меньше всего просматривают Lada, чаще всего Mercedes и BMW

In [20]:
df_temp = minimize_data_by_mark(df, 700)
labels, df_res = prepare_sankey_data(df_temp, ['Количество владельцев','Марка','Пробег тип'])
draw_sankey(labels, df_res, 'Марка - Пробег - Количество владельцев')